# Descripción de los datos

In [16]:
# importa las librerias requeridas
import pandas as pd
import os


In [23]:
# leer el archivo music_proyect_en.csv de la carpeta datasets y guardarlos en una variable
music_df = pd.read_csv('../datasets/raw/music_project_en.csv') 

In [24]:
# mostrar las primeras filas
print(music_df.head(10))

     userID                        Track            artist   genre  \
0  FFB692EC            Kamigata To Boots  The Mass Missile    rock   
1  55204538  Delayed Because of Accident  Andreas Rönnberg    rock   
2    20EC38            Funiculì funiculà       Mario Lanza     pop   
3  A3DD03C9        Dragons in the Sunset        Fire + Ice    folk   
4  E2DC1FAE                  Soul People        Space Echo   dance   
5  842029A1                       Chains          Obladaet  rusrap   
6  4CB90AA5                         True      Roman Messer   dance   
7  F03E1C1F             Feeling This Way   Polina Griffith   dance   
8  8FA1D3BE                     L’estate       Julia Dalia  ruspop   
9  E772D5C0                    Pessimist               NaN   dance   

        City        time        Day  
0  Shelbyville  20:28:33  Wednesday  
1  Springfield  14:07:09     Friday  
2  Shelbyville  20:58:07  Wednesday  
3  Shelbyville  08:37:09     Monday  
4  Springfield  08:34:34     Monday  
5

In [25]:
music_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63736 non-null  object
 2   artist    57512 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


In [27]:
print('Valores duplicados: ', music_df.duplicated().sum())

Valores duplicados:  3826


Contiene siete columnas. Almacenan los mismos tipos de datos: object.

Según la documentación:

' userID': identificador del usuario o la usuaria;
'Track': título de la canción;
'artist': nombre del artista;
'genre': género de la pista;
'City': ciudad del usuario o la usuaria;
'time': la hora exacta en la que se reprodujo la canción;
'Day': día de la semana.


Podemos ver tres problemas con el estilo en los encabezados de la tabla:

Algunos encabezados están en mayúsculas, otros en minúsculas.
Hay espacios en algunos encabezados.
Encanezados con varias palabras por lo que es necesario usar snake_case.

Observaciones: 

1.   ¿Qué tipo de datos tenemos a nuestra disposición en las filas? ¿Y cómo podemos entender lo que almacenan las columnas? 

Todos los datos son de tipo 'object' y a traves de la documentación y nombre de las columnas podemos determinar que representa cada columna

2.   ¿Hay suficientes datos para proporcionar respuestas a nuestra hipótesis o necesitamos más información? 

Tendriamos que determinar cuales son las ciudades en las que esta contemplado el estudio para poder responder esta pregunta, ya que si son para todo el mundo podriamos decir que los datos no son suficientes, sin embargo podemos delimitar nuestro estudio determinando las ciudades o la región, tambien estamos determinando que solo existe un solo conjunto de datos y no sabemos de donde provienen por lo que podriamos solicitar otras fuentes de datos para poder complementarlos.

3.   ¿Notaste algún problema en los datos, como valores ausentes, duplicados o tipos de datos incorrectos? 

En las columnas 'Track','artist' y 'genre' encontramos valores ausentes, y existen 3826 valores que son duplicados por lo que se deberian eliminar.

# Etapa 2 - Preprocesamiento de datos

El objetivo aquí es preparar los datos para que sean analizados. El primer paso es resolver cualquier problema con los encabezados. Luego podemos avanzar a los valores ausentes y duplicados


In [28]:
print(music_df.columns)

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')


Se cambiaran los encabezados de la tabla de acuerdo con las reglas del buen estilo:

* Todos los caracteres deben ser minúsculas.
* Elimina los espacios.
* Si el nombre tiene varias palabras, utiliza snake_case.

In [29]:
new_columns_name_lower = []# Bucle en los encabezados poniendo todo en minúsculas
for old_name in music_df.columns:
    name_lowered = old_name.lower()
    new_columns_name_lower.append(name_lowered) 
    
music_df.columns = new_columns_name_lower
print(music_df.columns)

Index(['  userid', 'track', 'artist', 'genre', '  city  ', 'time', 'day'], dtype='object')


In [30]:
new_columns_name_strip = []# Bucle en los encabezados eliminando los espacios
for old_name in music_df.columns:
    name_shipped = old_name.strip()
    new_columns_name_strip.append(name_shipped)
    
music_df.columns = new_columns_name_strip
print(music_df.columns)

Index(['userid', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')


In [33]:
new_column_s_c = {'userid':'user_id'}# Cambiar el nombre de la columna "userid"
music_df.rename(columns = new_column_s_c, inplace=True)
print(music_df.columns)

Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')


### Valores ausentes


In [34]:
print(music_df.isna().sum())# Calcular el número de valores ausentes

user_id       0
track      1343
artist     7567
genre      1198
city          0
time          0
day           0
dtype: int64


No todos los valores ausentes afectan a la investigación. Por ejemplo, los valores ausentes en track y artist no son cruciales. Simplemente puedes reemplazarlos con valores predeterminados como el string 'unknown' (desconocido).

Pero los valores ausentes en 'genre' pueden afectar la comparación entre las preferencias musicales de Springfield y Shelbyville. En la vida real, sería útil saber las razones por las cuales hay datos ausentes e intentar recuperarlos. Pero no tenemos esa oportunidad en este proyecto. Así que tendrás que:

* rellenar estos valores ausentes con un valor predeterminado;
* evaluar cuánto podrían afectar los valores ausentes a tus cómputos;

Reemplazar los valores ausentes en las columnas 'track', 'artist' y 'genre' con el string 'unknown'. Como mostramos anteriormente en las lecciones, la mejor forma de hacerlo es crear una lista que almacene los nombres de las columnas donde se necesita el reemplazo. Luego, utiliza esta lista e itera sobre las columnas donde se necesita el reemplazo haciendo el propio reemplazo.

In [35]:
# Bucle en los encabezados reemplazando los valores ausentes con 'unknown'
columns_to_replace = ['track','artist','genre']
for col in columns_to_replace:
    music_df[col].fillna('unknown', inplace = True)

/var/folders/2k/b1dxbcm15wl1lwf54_m3c9lm0000gn/T/ipykernel_4366/4178279495.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  music_df[col].fillna('unknown', inplace = True)


In [37]:
print(music_df.isna().sum())# Contar valores ausentes

user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64


### Duplicados

In [38]:
print(music_df.duplicated().sum())# Contar duplicados explícitos

3826


In [40]:
music_df = music_df.drop_duplicates().reset_index(drop = True)# Eliminar duplicados explícitos
print(music_df.duplicated().sum())# Comprobar de nuevo si hay duplicados

0


Ahora queremos deshacernos de los duplicados implícitos en la columna genre. Por ejemplo, el nombre de un género se puede escribir de varias formas. Dichos errores también pueden afectar al resultado.

Para hacerlo, primero mostremos una lista de nombres de género únicos, ordenados en orden alfabético. Para ello:

Extrae la columna genre del DataFrame.
Llama al método que devolverá todos los valores únicos en la columna extraída.

In [41]:
unique_values = music_df['genre'].unique()# Inspeccionar los nombres de géneros únicos
unique_values.sort() #no devolvio el listado en orden alfabetico asi que le pase el metodo sort()

print(unique_values)
print(music_df['genre'].nunique())

['acid' 'acoustic' 'action' 'adult' 'africa' 'afrikaans' 'alternative'
 'ambient' 'americana' 'animated' 'anime' 'arabesk' 'arabic' 'arena'
 'argentinetango' 'art' 'audiobook' 'avantgarde' 'axé' 'baile' 'balkan'
 'beats' 'bigroom' 'black' 'bluegrass' 'blues' 'bollywood' 'bossa'
 'brazilian' 'breakbeat' 'breaks' 'broadway' 'cantautori' 'cantopop'
 'canzone' 'caribbean' 'caucasian' 'celtic' 'chamber' 'children' 'chill'
 'chinese' 'choral' 'christian' 'christmas' 'classical' 'classicmetal'
 'club' 'colombian' 'comedy' 'conjazz' 'contemporary' 'country' 'cuban'
 'dance' 'dancehall' 'dancepop' 'dark' 'death' 'deep' 'deutschrock'
 'deutschspr' 'dirty' 'disco' 'dnb' 'documentary' 'downbeat' 'downtempo'
 'drum' 'dub' 'dubstep' 'eastern' 'easy' 'electronic' 'electropop' 'emo'
 'entehno' 'epicmetal' 'estrada' 'ethnic' 'eurofolk' 'european'
 'experimental' 'extrememetal' 'fado' 'film' 'fitness' 'flamenco' 'folk'
 'folklore' 'folkmetal' 'folkrock' 'folktronica' 'forró' 'frankreich'
 'französisch' 

Busca en la lista para encontrar duplicados implícitos del género hiphop. Estos pueden ser nombres escritos incorrectamente o nombres alternativos para el mismo género.

Verás los siguientes duplicados implícitos:

* hip
* hop
* hip-hop


Para deshacernos de ellos, se creará una función llamada replace_wrong_genres() con dos parámetros:

* wrong_genres=: esta es una lista que contiene todos los valores que necesitas reemplazar.
* correct_genre=: este es un string que vas a utilizar como reemplazo.

Como resultado, la función debería corregir los nombres en la columna 'genre' de la tabla df, es decir, remplazar cada valor de la lista wrong_genres por el valor en correct_genre.


In [42]:
def replace_wrong_generes(wrong_genres,correct_genre):# Función para reemplazar duplicados implícitos
    for value in wrong_genres:
        music_df['genre'] = music_df['genre'].replace(value,correct_genre)
    return music_df

In [43]:
duplicates_value = ['hip','hop','hip-hop']# Eliminar duplicados implícitos
correct_value = 'hiphop'

music_df = replace_wrong_generes(duplicates_value,correct_value)

In [45]:
uniq_values = music_df['genre'].unique()
uniq_values.sort()
print(uniq_values)# Comprobación de duplicados implícitos
print(music_df['genre'].nunique())

['acid' 'acoustic' 'action' 'adult' 'africa' 'afrikaans' 'alternative'
 'ambient' 'americana' 'animated' 'anime' 'arabesk' 'arabic' 'arena'
 'argentinetango' 'art' 'audiobook' 'avantgarde' 'axé' 'baile' 'balkan'
 'beats' 'bigroom' 'black' 'bluegrass' 'blues' 'bollywood' 'bossa'
 'brazilian' 'breakbeat' 'breaks' 'broadway' 'cantautori' 'cantopop'
 'canzone' 'caribbean' 'caucasian' 'celtic' 'chamber' 'children' 'chill'
 'chinese' 'choral' 'christian' 'christmas' 'classical' 'classicmetal'
 'club' 'colombian' 'comedy' 'conjazz' 'contemporary' 'country' 'cuban'
 'dance' 'dancehall' 'dancepop' 'dark' 'death' 'deep' 'deutschrock'
 'deutschspr' 'dirty' 'disco' 'dnb' 'documentary' 'downbeat' 'downtempo'
 'drum' 'dub' 'dubstep' 'eastern' 'easy' 'electronic' 'electropop' 'emo'
 'entehno' 'epicmetal' 'estrada' 'ethnic' 'eurofolk' 'european'
 'experimental' 'extrememetal' 'fado' 'film' 'fitness' 'flamenco' 'folk'
 'folklore' 'folkmetal' 'folkrock' 'folktronica' 'forró' 'frankreich'
 'französisch' 

In [46]:
music_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61253 entries, 0 to 61252
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  61253 non-null  object
 1   track    61253 non-null  object
 2   artist   61253 non-null  object
 3   genre    61253 non-null  object
 4   city     61253 non-null  object
 5   time     61253 non-null  object
 6   day      61253 non-null  object
dtypes: object(7)
memory usage: 3.3+ MB


In [47]:
music_df.to_csv('../datasets/clean/music_project_clean.csv', index=False)

Obervaciones:

Para poder determinar bien si los datos son duplicados deberiamos conocer cada unos de los generos que existen en la lista, ya que aunque en el caso anterior hiphop, se puede escribir de diferentes maneras y es mas probable que exista un error, hay otro genero que podriamos decir que es el mismo, en el caso de 'latin' y 'latino' en la documentación nos deberia decir si son dos generos distintos o en esta caso tambien son duplicados y poder replazaro por el correcto.
Podriamos decir que los generos de musica son algo subjetivos ya que por ejemplo en el caso de la electronica('electronic') podriamos que tiene varios subgeneros como el techo, dance, house, por lo que tendriamos que definir estos aspectos. Pero para no desviarnos del tema, en este caso estamos buscando duplicados que puedan tener un error de escritura como el hiphop, para los demas generos podemos observar que no es el caso.
